In [1]:
import casadi as ca
import numpy as np
import sympy as sp

## Orthogonal collocation with finite elements

reference: 
1. L.T. Biegler. Nonlinear Programming: Concepts, Algorithms, and Applications to Chemical Processes. SIAM, 2010.
2. do-mpc: https://www.do-mpc.com/en/latest/theory_orthogonal_collocation.html#biegler2010
3. Dr. Abebe Geletu. Introduction to Differential Algebraic Equations

Since the dynamic model equations are formulated as ODE or DAE, the continuous time model should be either directly or indirectly incorporated into the NLP solver. Orthogonal collocation on finite elements as a direct method, which fully discretizes the continuous-time variable and tranforms the infinite-dimensional optimization problem to finite-dimensional problem, is considered below.

The basic idea of this approach is to use a polynomial to collocate the target function $z(t)$, which is originally given in the ODE form $\frac{dz}{dt} = f(z(t),t)$. This approach splits the domain of ODE into several small (equal-length) time intervals, called element. Within each element, the same number of collocations points are chosen. In the following, the approach looking for the appropriate collocation points will be introduced.

In [11]:
#TODO: add a fig.

### Collocation polynomial

The solution of ODE is approximated by the interpolation polynomial in the Lagrange form: $z^{K}_{i}(t)=\sum_{j=0}^{K} \ell_{j}(\tau) z_{i j}$, where $\ell_{j}(\tau)=\prod_{k=0, \neq j}^{K} \frac{\left(\tau-\tau_{k}\right)}{\left(\tau_{j}-\tau_{k}\right)}$. The polynomial could be equivalently represented with power series of order $k+1$: $z^{K}(t)=\alpha_{0}+\alpha_{1} t+\alpha_{2} t^{2}+\cdots+\alpha_{K} t^{K}$, where $t=t_{i-1}+h_{i} \tau$ with $t \in\left[t_{i-1}, t_{i}\right]$ and $\tau \in[0,1]$. The purpose of the time domain projection is to apply a standard method to choose the value of the nominal time variable $\tau$.
Note that $z_{ij}$ represents the $j$-th state in the $i$-th element. 

### Constraints

Since the derivatives of polynomial approximation at each collocation point is required to equal to the original ODE at the chosen points. This leads to the following collocation equations:
$\frac{d z^{K}}{d t}\left(t_{i k}\right)=f\left(z^{K}\left(t_{i k}\right), t_{i k}\right), \quad k=1, \ldots, K$. We substitue here $t$ with $\tau$, the collocation equations become $\sum_{j=0}^{K} z_{i j} \frac{d \ell_{j}\left(\tau_{k}\right)}{d \tau}=h_{i} f\left(z_{i k}, t_{i k}\right), \quad k=1, \ldots, K$.
These collocation contraints provide $K$ equations for $K+1$ variables within each element. Therefore, more contunuity constraints should be applied to enforce the continuity of the state profiles across element boundaries: 
$\begin{aligned} z_{i+1,0} &=\sum_{j=0}^{K} \ell_{j}(1) z_{i j}, \quad i=1, \ldots, N-1, \\ z_{f} &=\sum_{j=0}^{K} \ell_{j}(1) z_{N j}, \quad z_{1,0}=z_{0} \end{aligned}$

### Decision of collocation points

Till now, we have still insufficient knowledge about the position of collocation points. The basic idea behind choosing $\tau_j$ is to let the numerical solution approximate the implicit integral $z\left(t_{i}\right)=z\left(t_{i-1}\right)+\int_{t_{i-1}}^{t_{i}} f(z(t), t) d t$. The numerical is represented by the quadrature formula $z\left(t_{i}\right)=z\left(t_{i-1}\right)+\sum_{j=1}^{K} \omega_{j} h_{i} f\left(z\left(t_{i j}\right), t_{i j}\right), \quad t_{i j}=t_{i-1}+h_{i} \tau_{j}$.

The theorem of **accuracy of Gaussian Quadrature** gives the exact condition to choose the collocation points - collocations points $\tau_j$ are the root of the shifted Gauss-Legendre polynomial $P_K(\tau)$ with degree $K$. As alternatives, Gauss–Radau collocation or Gauss–Lobatto collocation could also be applied.

After inserting every acquired parameters into the collocation approximation $z_i^{K}(t)=\sum_{j=0}^{K} \ell_{j}(\tau) z_{i j}$, the work's done!

### Implementation

Orthogonal collocation on finite over finite elements in time order 4 (K = 3)

### First step: Formulate the interpolation polynomial

In [12]:
def lagrange_poly_fn(j,tau_vec):
    '''
    This function formulate the Lagrange polynomial for a given collocation point.
    
    Args:
        j: Index of of the selected collocation point in the tau vector.
        tau_vec: a vector in SX. Length of tau_vec is equal to K + 1 (tau_0 included).
    
    Return: 
        l_fn: casadi function: tau -> l_j(tau)
        It is a scalar function of time instant tau returns the Lagrange polynomial of a given collocation.
    '''
    N_tau = np.shape(tau_vec)[0] #  K + 1
    tau = ca.SX.sym("tau",1)
    
    l_j = 1
    for k in range(N_tau):
        if k != j:
            l_j = l_j * (tau - tau_vec[k]) / (tau_vec[j] - tau_vec[k]) 
    l_fn = ca.Function("lagrange_poly_" + str(j),[tau],[l_j])
    return l_fn
    
    

In [3]:
def collocation_equation(dae, N, tau_vec, state_init, t_begin=0.0,t_end=1.0):
    '''
    This function formulate the system of collocation equations, so as to identify the state x_ij and z_ij at each collocation points.
    
    TODO: 1. only for ode right now. Either write a wrapper or vertically concatenate algebriac equations.
    2. If this program is used in other subjects, reformulate the return. It is not neaty to return the defined variable.
    Solution to this problem: Either take z_var and x_var as input defined in the class or return a casadi function.
    3. the time step h is defined a constant here. In some paper step lengths are defined as variables. Make some changes if neccessary. 
    4. Support Radau and Lobatto collocation besides the applied Legendre collocation. 
    
    Args:
        dae: ODE/DAE function in casadi function (Nt x Nx x Nz -> Nx x Nz).
        N: The number of Elements.
        tau_vec: List of tau vector: tau_0, tau_1 ... tau_K (length K+1).
        state_init: Initial state as boundary of the initial value problem.
        t_begin: Start time of the collocation.
        t_end End time of the collocation.
    '''
    Nx = dae.sx_in()[1].shape[0]  # Number of differential variables (dae.sx_in()[0].shape[0] is t).
    Nz = dae.sx_in()[2].shape[0]  # Number of algebraic variables.
    
    N_tau = np.shape(tau_vec)[0] #  K + 1.
    N_element = N
    
    h = (t_end - t_begin) / N_element
    # Form (Nx + Nz) x (N_element x N_tau) equations to determing the (Nx + Nz) x (N_element x N_tau) unknowns.
    
    # Generate matrix for each variable.
    # Each element in the list is a variable matrix of size (Nx + Nz) x (N_element x N_tau).
    # TODO: find out if it is better to form a tensor.
    x_var = ca.SX.sym('x', Nx, N_element * N_tau)   
    z_var = ca.SX.sym('z', Nz, N_element * N_tau)
    # An extra column for the end state
    x_f_var = ca.SX.sym('x_f', Nx)
    
    tau_var = ca.SX.sym('tau',1) 
    
    # Get Lagrange polynomial and its time derivative of tau.
    l_fn = []   # Lagrange polynomial in casadi function.
    ld = []    # Derivative of Lagrange polynomial in casadi.SX.
    ld_fn = []    # Derivative of Lagrange polynomial in casadi fucntion.
    col_eq = []    # Collocation equation in casadi.SX.
    
    # Get prepared with the Lagrange polynomials.
    for i in range(N_tau):
        l_fn.append(lagrange_poly_fn(i,tau_vec))
        ld_temp = ca.jacobian(l_fn[i](tau_var), tau_var)
        ld.append(ld_temp)
        ld_fn.append(ca.Function("ld" + str(i),[tau_var] ,[ld_temp] ))
    
    # Interpolation polynomials in Lagrange form.
    x_poly = []    # \Sigma_{j=0}^{K} z_{ij}l_j(\tau_k)
    x_poly_d = []    # l_j time derivative of the interpolation polynomials.
    # Iterate over all the elements.
    for i in range(N_element):
        # Within a element, sum the polynomial function of each collocation point up. 
        t_i = t_begin + i * h
        interp_end_temp = 0
        for k in range(N_tau-1): 
            interp_temp = 0
            interp_d_temp = 0
            for j in range(N_tau):
                # Interpolation polinomial and its derivatives in casadi.SX.
                # k+1 since starting from tau_1 in each element. (tau_0 should not be iterated).
                interp_temp += x_var[:,i*N_element + j] * l_fn[j](tau_vec[k+1])
                interp_d_temp += x_var[:,i*N_element + j] * ld_fn[j](tau_vec[k+1])
                
                # Continuity condition across the boundary of elements: calculate the end value only once in each element.
                if k == 0: 
                    interp_end_temp += x_var[:,i*N_element + j] * l_fn[j](1)
                    
            x_poly.append(interp_temp)
            x_poly_d.append(interp_d_temp)
            # Formulate collocation equation as lefthand-sided - righthand-sided. 
            t_ik = h * tau_vec[k+1] + t_i    # linear projection, from tau to t.
            col_eq = ca.vertcat(col_eq, (interp_d_temp - h * dae(t_ik, x_var[:,i*N_element + k+1], z_var[:,i*N_element + k+1])))  # k+1 since fisrt condition at tau_1   
            # Formulate equation for the continuity condition.
            if k == 0 and (i < N_element - 1):
                col_eq = ca.vertcat(col_eq,(interp_end_temp - x_var[:,(i + 1)*N_element]))
            if k == 0 and i == N_element - 1:
                col_eq = ca.vertcat(col_eq,(interp_end_temp - x_f_var))
    col_eq = ca.vertcat(col_eq,(x_var[:,0] - state_init))
    
    return ca.horzcat(x_var, x_f_var), z_var, col_eq

### Example 1 
For debug: $\dot{x} = x$, analytical solution is $x = x_0e^t$

In [4]:
def ode_ca(t, x, z):
    '''
    1-dimensional toy system
    '''
    # Parameter konfiguration
    rhs = [x
           ]

    return ca.vertcat(*rhs)

Nx = 1    # x dimension: 1
Nt = 1    # t dimension: 1
Nz = 0

x = ca.SX.sym('x', Nx)
t = ca.SX.sym('t', Nt)
z = ca.SX.sym('z', Nz)

# Construct a casadi function for the ODE
fn_ca = ca.Function("dae_func", [t, x,z], [ode_ca(t,x,z)])


In [5]:
# Acquire tau from the look-up table for Gauss-Legendre roots as collocation points. 
tau_vec = [0.0000, 0.112702, 0.50000, 0.887298]
x0 = [1.0]


x_var, z_var, eq = collocation_equation(fn_ca,1,tau_vec,x0)
x_var = ca.reshape(x_var, -1,1)


g = ca.Function('g',[x_var],[eq])
G = ca.rootfinder('G','newton',g)

G([0,0,0,0,0])

DM([1, 1.12006, 1.64789, 2.42924, 2.71831])

Validate the return of $\dot{l}_j$ using sympy

In [6]:
tau = sp.symbols('tau')
tau_vec = [0.0000, 0.112702, 0.50000, 0.887298]

j = 1
l_0 = (tau - tau_vec[0]) / (tau_vec[j] - tau_vec[0]) * (tau - tau_vec[2]) / (tau_vec[j] - tau_vec[2]) * (tau - tau_vec[3]) / (tau_vec[j] - tau_vec[3]) 
l_0_d = sp.diff(l_0,tau)
l_0_d = sp.expand(l_0_d)
l_0_d_fn = sp.lambdify(tau,l_0_d)
l_0_d_fn(1)

19.78829586306479

### Example 2: double integrator
Test dynamical system: $\begin{aligned} \dot{x}_1 &= x_2 \\ \dot{x}_2 &= u\end{aligned}$

Analytical solution: $\begin{aligned} x_1 &= x_{10} + x_{20}t + \frac{1}{2}ut^{2} \\ x_2 &= x_{20} + ut\end{aligned}$

In [7]:
def ode_ca(t, x, z):
    '''
    Double integrator
    
    Args:
        t: Current time.
        x: Current value (list or numpy array). 
        
    Returns: First order ODE in casadi SX col vector .
    '''
    # Parameter konfiguration
    u = 1.0
    rhs = [x[1],
           u
           ]

    return ca.vertcat(*rhs)

Nx = 2    # x dimension: 2
Nt = 1    # t dimension: 1
Nz = 0

x = ca.SX.sym('x', Nx)
t = ca.SX.sym('t', Nt)
z = ca.SX.sym('z', Nz)
# print([t, x, z])
# Construct a casadi function for the ODE
fn_ca = ca.Function("dae_func", [t, x,z], [ode_ca(t,x,z)])


In [8]:
tau_vec = [0.0000, 0.112702, 0.50000, 0.887298]
x0 = [1.0, 2.0]


x_var, z_var, eq = collocation_equation(fn_ca,1,tau_vec,x0)
x_var = ca.reshape(x_var, -1,1)

 
g = ca.Function('g',[x_var],[eq])
G = ca.rootfinder('G','newton',g)

G([1,1,1,1,1,1,1,1,1,1])

DM([1, 2, 1.23175, 2.1127, 2.125, 2.5, 3.16824, 2.8873, 3.5, 3])

### Example 3: Demonstration example from the book Nonlinear programming written by Lorenz T. Biegler
Test dynamical system: $\frac{d z}{d t}=z^{2}-2 z+1, \quad z(0)=-3$

Analytical solution: $z(t)=(4 t-3) /(4 t+1)$



In [9]:
def ode_ca(t, x, z):
    '''
    Demonstration of Orthogonal Collocation
    '''
    # Parameter konfiguration
    rhs = [x[0]**2 - 2*x[0]+1
           ]

    return ca.vertcat(*rhs)

Nx = 1    # x dimension: 1
Nt = 1    # t dimension: 1
Nz = 0

x = ca.SX.sym('x', Nx)
t = ca.SX.sym('t', Nt)
z = ca.SX.sym('z', Nz)
# print([t, x, z])
# Construct a casadi function for the ODE
fn_ca = ca.Function("dae_func", [t, x,z], [ode_ca(t,x,z)])
print(fn_ca)

dae_func:(i0,i1,i2[0])->(o0) SXFunction


In [10]:
tau_vec = [0.0000, 0.112702, 0.50000, 0.887298]
x0 = [-3]

x_var, z_var, eq = collocation_equation(fn_ca,1,tau_vec,x0)
x_var = ca.reshape(x_var, -1,1)
 
g = ca.Function('g',[x_var],[eq])
G = ca.rootfinder('G','newton',g)

G([0,0,0,0,0])

DM([-3, -1.88606, -0.199567, 0.0490687, 0.204433])

A general definition question: order is its order is conventionally the degree +1